In [ ]:
from model import get_tb_model

In [ ]:
model_config = {
    "start_time": 1850,
    "end_time": 2050,
    "seed": 100,   
}

In [ ]:
studies_dict = {
    "study_1": {
        "pop_size": 2.e6,
    },
    "study_2": {
        "pop_size": 1.e6,
    }    
}

In [ ]:
model = get_tb_model(model_config, studies_dict)

In [ ]:
model.compartments

In [ ]:
model.get_input_parameters()

In [ ]:
params = {
    # Study-specific parameters
    'transmission_rateXstudy_1': 10,
    'transmission_rateXstudy_2': 12,

    'activation_rate_early': .1,
    'activation_rate_late': .001,
    'stabilisation_rate': 1.,

    'current_passive_detection_rate': 1.,

    # Universal parameters
    'rr_reinfection_latent_late': .2,
    'rr_reinfection_recovered': 1.,
    'self_recovery_rate': .2,
    'tb_death_rate': .2,
    'tx_duration': .5,
    'tx_prop_death': .04
}
model.run(params)

In [ ]:
do_df = model.get_derived_outputs_df()
do_df[['study_1Xltbi_prop', 'study_2Xltbi_prop']].plot()
do_df[['study_1Xpopulation', 'study_2Xpopulation']].plot()

In [ ]:
model.flows